In [31]:
from natsort import natsorted
from collections import defaultdict

In [336]:
class XdcParser:
    def __init__(self, fName):
        ''' fName is a Xilinx Master .xdc file from their website '''
        with open(fName) as f:
            ls = f.readlines()

        self.props = defaultdict(dict)
        for l in ls:
            l = l.strip()
            if l.startswith('#'):
                continue
            ps = l.split()
            if not ps[0] == 'set_property':
                continue
            if ps[3] == '[get_ports':
                self.props[ps[4].replace(']', '')][ps[1]] = ps[2]
            else:
                print('parser is too dumb for this:', ps)
        
        self.keys = natsorted(props.keys())
        
    def filterKeys(self, regex):
        pp = re.compile(regex)
        return [x for x in self.keys if pp.fullmatch(x)]

    def getGpios(self, liteName='user_led', xilRegex='GPIO_LED.*'):
        for i, k in enumerate(p.filterKeys(xilRegex)):
            print('    ("{:}", {:}, Pins("{:}"), IOStandard("{:}")),'.format(
                liteName,
                i, 
                p.props[k]["PACKAGE_PIN"],
                p.props[k]["IOSTANDARD"]
            ))
    
    def getConnector(self, fmcName='FMC1_HPC', nameReplace=None):
        fmcName_ = fmcName + '_'
        print('    ("{:}", {{'.format(fmcName))
        for k in filter(lambda x: x.startswith(fmcName_), self.keys):
            pinName = k.replace(fmcName_, '')
            if nameReplace:
                for rep in nameReplace:
                    pinName = pinName.replace(*rep)
            print('        "{:}": "{:}",'.format(pinName, props[k]['PACKAGE_PIN']))
        print('    }),')
        
    def getSubSignal(self, liteName, xilRegex, maxPins=None):
        '''
        liteName='rx_p'
        xilRegex: 'PCIE_RX[0-9]_P'
        maxPins: upper limit on the number of pins in the list
        '''
        pins = ''
        ks = p.filterKeys(xilRegex)
        if maxPins is not None:
            ks = ks[:maxPins]
            if len(ks) < maxPins:
                raise RuntimeError("Not enough matches!")
        for k in ks:
            pins += p.props[k]["PACKAGE_PIN"] + ' '
        pins = pins[:-1]
        try:
            # Let's hope all the IO standards are the same :p
            ios = p.props[k]["IOSTANDARD"]
            ios = ', IOStandard("{:}")'.format(ios)
        except:
            ios = ''
            pass
        print('Subsignal("{:}", Pins("{:}"){:}),'.format(
            liteName,
            pins,
            ios
        ))
        
    def getGroup(self, name, tuples):
        print('    ("{:}", 0,'.format(name))
        for t in tuples:
            print('        ', end='')
            p.getSubSignal(*t)
        print('    ),')

In [337]:
p = XdcParser('./VC707_rev_2.0.ucf.xdc')

In [338]:
p.getGroup('eth', (
    ('rst_n', 'PHY_RESET_LS'),
    ('int_n', 'PHY_INT_LS'),
    ('mdio', 'PHY_MDIO_LS'),
    ('mdc', 'PHY_MDC_LS'),
    ('rx_p', 'SGMII_RX_P'),
    ('rx_n', 'SGMII_RX_N'),
    ('tx_p', 'SGMII_TX_P'),
    ('tx_n', 'SGMII_TX_N'),
))
p.getGroup('sgmii_clock', (
    ('p', 'SGMIICLK_Q0_P'),
    ('n', 'SGMIICLK_Q0_N'),
))

    ("eth", 0,
        Subsignal("rst_n", Pins("AJ33"), IOStandard("LVCMOS18")),
        Subsignal("int_n", Pins("AL31"), IOStandard("LVCMOS18")),
        Subsignal("mdio", Pins("AK33"), IOStandard("LVCMOS18")),
        Subsignal("mdc", Pins("AH31"), IOStandard("LVCMOS18")),
        Subsignal("rx_p", Pins("AM8")),
        Subsignal("rx_n", Pins("AM7")),
        Subsignal("tx_p", Pins("AN2")),
        Subsignal("tx_n", Pins("AN1")),
    ),
    ("sgmii_clock", 0,
        Subsignal("p", Pins("AH8")),
        Subsignal("n", Pins("AH7")),
    ),


In [339]:
for i in [1, 2, 4, 8]:
    p.getGroup('pcie_x{:}'.format(i), (
        ('rst_n', 'PCIE_PERST_.*'),
        ('clk_p', 'PCIE_CLK_QO_P'),
        ('clk_n', 'PCIE_CLK_QO_N'),
        ('rx_p', 'PCIE_RX[0-7]_P', i),
        ('rx_n', 'PCIE_RX[0-7]_N', i),
        ('tx_p', 'PCIE_TX[0-7]_P', i),
        ('tx_n', 'PCIE_TX[0-7]_N', i),
    ))

    ("pcie_x1", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4")),
        Subsignal("rx_n", Pins("Y3")),
        Subsignal("tx_p", Pins("W2")),
        Subsignal("tx_n", Pins("W1")),
    ),
    ("pcie_x2", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4 AA6")),
        Subsignal("rx_n", Pins("Y3 AA5")),
        Subsignal("tx_p", Pins("W2 AA2")),
        Subsignal("tx_n", Pins("W1 AA1")),
    ),
    ("pcie_x4", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4 AA6 AB4 AC6")),
        Subsignal("rx_n", Pins("Y3 AA5 AB3 AC5")),
        Subsignal("tx_p", Pins("W2 AA2 AC2 AE2")),
    

In [308]:
p.getGroup('clk200', (
    ('p', 'SYSCLK_P'),
    ('n', 'SYSCLK_N'),
))
p.getGpios('user_sma_clock_p', 'USER_SMA_CLOCK_P')
p.getGpios('user_sma_clock_n', 'USER_SMA_CLOCK_N')
p.getGroup('user_sma_clock', (
    ('p', 'USER_SMA_CLOCK_P'),
    ('n', 'USER_SMA_CLOCK_N'),
))
p.getGpios('user_clock_p', 'USER_CLOCK_P')
p.getGpios('user_clock_n', 'USER_CLOCK_N')
p.getGroup('user_sma_mgt_tx', (
    ('p', 'SMA_MGT_TX_P'),
    ('n', 'SMA_MGT_TX_N'),
))
p.getGroup('user_sma_mgt_rx', (
    ('p', 'SMA_MGT_RX_P'),
    ('n', 'SMA_MGT_RX_N'),
))
p.getGroup('user_sma_mgt_refclk', (
    ('p', 'SMA_MGT_REFCLK_P'),
    ('n', 'SMA_MGT_REFCLK_N'),
))
p.getGroup('si5324', (
    ('rst_n', 'SI5324_RST_LS'),
    ('int', 'SI5324_INT_ALM_LS'),
))
p.getGroup('si5324_clkin', (
    ('p', 'REC_CLOCK_C_P'),
    ('n', 'REC_CLOCK_C_N'),
))

("clk200", 0,
    Subsignal("p", Pins("E19"), IOStandard("LVDS")),
    Subsignal("n", Pins("E18"), IOStandard("LVDS")),
),
("user_sma_clock_p", 0, Pins("AJ32"), IOStandard("LVCMOS18")),
("user_sma_clock_n", 0, Pins("AK32"), IOStandard("LVCMOS18")),
("user_sma_clock", 0,
    Subsignal("p", Pins("AJ32"), IOStandard("LVCMOS18")),
    Subsignal("n", Pins("AK32"), IOStandard("LVCMOS18")),
),
("user_clock_p", 0, Pins("AK34"), IOStandard("LVDS")),
("user_clock_n", 0, Pins("AL34"), IOStandard("LVDS")),
("user_sma_mgt_tx", 0,
    Subsignal("p", Pins("AP4")),
    Subsignal("n", Pins("AP3")),
),
("user_sma_mgt_rx", 0,
    Subsignal("p", Pins("AN6")),
    Subsignal("n", Pins("AN5")),
),
("user_sma_mgt_refclk", 0,
    Subsignal("p", Pins("AK8")),
    Subsignal("n", Pins("AK7")),
),
("si5324", 0,
    Subsignal("rst_n", Pins("AT36"), IOStandard("LVCMOS18")),
    Subsignal("int", Pins("AU34"), IOStandard("LVCMOS18")),
),
("si5324_clkin", 0,
    Subsignal("p", Pins("AW32"), IOStandard("LVCMOS18")),
   

In [309]:
p.getGpios('cpu_reset', 'CPU_RESET')
p.getGpios('user_led', 'GPIO_LED_[0-9]_LS')
p.getGpios('user_dip_btn', 'GPIO_DIP_SW[0-9]')
p.getGpios('user_btn_c', 'GPIO_SW_C')
p.getGpios('user_btn_n', 'GPIO_SW_N')
p.getGpios('user_btn_e', 'GPIO_SW_E')
p.getGpios('user_btn_s', 'GPIO_SW_S')
p.getGpios('user_btn_w', 'GPIO_SW_W')
p.getGroup('rotary', (
    ('a', 'ROTARY_INCA'),
    ('b', 'ROTARY_INCB'),
    ('push', 'ROTARY_PUSH'),
))
p.getGpios('user_sma_gpio_p', 'USER_SMA_GPIO_P')
p.getGpios('user_sma_gpio_n', 'USER_SMA_GPIO_N')
p.getGroup('lcd', (
    ('db', 'LCD_DB[4-7]_LS'),
    ('rs', 'LCD_RS_LS'),
    ('rw', 'LCD_RW_LS'),
    ('e', 'LCD_E_LS'),
))
p.getGroup('i2c', (
    ('scl', 'IIC_SCL_MAIN_LS'),
    ('sda', 'IIC_SDA_MAIN_LS'),
))
p.getGpios('i2c_mux_reset', 'IIC_MUX_RESET_B_LS')
p.getConnector("XADC", nameReplace=(('N_R', '_N'),('P_R', '_P')))
p.getGroup('serial', (
    ('rx', 'USB_UART_RX'),
    ('rts', 'USB_UART_RTS'),
    ('tx', 'USB_UART_TX'),
    ('cts', 'USB_UART_CTS'),
))

("cpu_reset", 0, Pins("AV40"), IOStandard("LVCMOS18")),
("user_led", 0, Pins("AM39"), IOStandard("LVCMOS18")),
("user_led", 1, Pins("AN39"), IOStandard("LVCMOS18")),
("user_led", 2, Pins("AR37"), IOStandard("LVCMOS18")),
("user_led", 3, Pins("AT37"), IOStandard("LVCMOS18")),
("user_led", 4, Pins("AR35"), IOStandard("LVCMOS18")),
("user_led", 5, Pins("AP41"), IOStandard("LVCMOS18")),
("user_led", 6, Pins("AP42"), IOStandard("LVCMOS18")),
("user_led", 7, Pins("AU39"), IOStandard("LVCMOS18")),
("user_dip_btn", 0, Pins("AV30"), IOStandard("LVCMOS18")),
("user_dip_btn", 1, Pins("AY33"), IOStandard("LVCMOS18")),
("user_dip_btn", 2, Pins("BA31"), IOStandard("LVCMOS18")),
("user_dip_btn", 3, Pins("BA32"), IOStandard("LVCMOS18")),
("user_dip_btn", 4, Pins("AW30"), IOStandard("LVCMOS18")),
("user_dip_btn", 5, Pins("AY30"), IOStandard("LVCMOS18")),
("user_dip_btn", 6, Pins("BA30"), IOStandard("LVCMOS18")),
("user_dip_btn", 7, Pins("BB31"), IOStandard("LVCMOS18")),
("user_btn_c", 0, Pins("AV39"), 

In [310]:
p.getGroup('hdmi', (
    ('d', 'HDMI_R_D\d+', 36),
    ('de', 'HDMI_R_DE'),
    ('clk', 'HDMI_R_CLK'),
    ('vsync', 'HDMI_R_VSYNC'),
    ('hsync', 'HDMI_R_HSYNC'),
    ('int', 'HDMI_INT'),
    ('spdif', 'HDMI_R_SPDIF'),
    ('spdif_out', 'HDMI_SPDIF_OUT_LS')
))

("hdmi", 0,
    Subsignal("d", Pins("AM22 AL22 AJ20 AJ21 AM21 AL21 AK22 AJ22 AL20 AK20 AK23 AJ23 AN21 AP22 AP23 AN23 AM23 AN24 AY24 BB22 BA22 BA25 AY25 AY22 AY23 AV24 AU24 AW21 AV21 AT24 AR24 AU21 AT21 AW22 AW23 AV23"), IOStandard("LVCMOS18")),
    Subsignal("de", Pins("AP21"), IOStandard("LVCMOS18")),
    Subsignal("clk", Pins("AU23"), IOStandard("LVCMOS18")),
    Subsignal("vsync", Pins("AT22"), IOStandard("LVCMOS18")),
    Subsignal("hsync", Pins("AU22"), IOStandard("LVCMOS18")),
    Subsignal("int", Pins("AM24"), IOStandard("LVCMOS18")),
    Subsignal("spdif", Pins("AR23"), IOStandard("LVCMOS18")),
    Subsignal("spdif_out", Pins("AR22"), IOStandard("LVCMOS18")),
),


In [326]:
for n, i in (('ddram', 1), ('ddram_dual_rank', 2)):
    p.getGroup(n, (
        ('a', 'DDR3_A\d+', 16),
        ('ba', 'DDR3_BA\d', 3),
        ('ras_n', 'DDR3_RAS_B'),
        ('cas_n', 'DDR3_CAS_B'),
        ('we_n', 'DDR3_WE_B'),
        ('cs_n', 'DDR3_S\d_B', i), # hope this is right ???
        ('dm', 'DDR3_DM\d', 8),
        ('dq', 'DDR3_D\d+', 64),
        ('dqs_p', 'DDR3_DQS\d_P', 8),
        ('dqs_n', 'DDR3_DQS\d_N', 8),
        ('clk_p', 'DDR3_CLK\d_P', i),
        ('clk_n', 'DDR3_CLK\d_N', i),
        ('cke', 'DDR3_CKE\d', i),
        ('odt', 'DDR3_ODT\d', i),
        ('reset_n', 'DDR3_RESET_B'),   
    ))

("ddram", 0,
    Subsignal("a", Pins("A20 B19 C20 A19 A17 A16 D20 C18 D17 C19 B21 B17 A15 A21 F17 E17"), IOStandard("SSTL15")),
    Subsignal("ba", Pins("D21 C21 D18"), IOStandard("SSTL15")),
    Subsignal("ras_n", Pins("E20"), IOStandard("SSTL15")),
    Subsignal("cas_n", Pins("K17"), IOStandard("SSTL15")),
    Subsignal("we_n", Pins("F20"), IOStandard("SSTL15")),
    Subsignal("cs_n", Pins("J17"), IOStandard("SSTL15")),
    Subsignal("dm", Pins("M13 K15 F12 A14 C23 D25 C31 F31"), IOStandard("SSTL15")),
    Subsignal("dq", Pins("N14 N13 L14 M14 M12 N15 M11 L12 K14 K13 H13 J13 L16 L15 H14 J15 E15 E13 F15 E14 G13 G12 F14 G14 B14 C13 B16 D15 D13 E12 C16 D16 A24 B23 B27 B26 A22 B22 A25 C24 E24 D23 D26 C25 E23 D22 F22 E22 A30 D27 A29 C28 D28 B31 A31 A32 E30 F29 F30 F27 C30 E29 F26 D30"), IOStandard("SSTL15")),
    Subsignal("dqs_p", Pins("N16 K12 H16 C15 A26 F25 B28 E27"), IOStandard("DIFF_SSTL15")),
    Subsignal("dqs_n", Pins("M16 J12 G16 C14 A27 E25 B29 E28"), IOStandard("DIFF_SSTL15"))

In [335]:
p.getGroup('sfp', (
    ('txp', 'SFP_TX_P'),
    ('txn', 'SFP_TX_N'),
    ('rxp', 'SFP_RX_P'),
    ('rxn', 'SFP_RX_N')
))
p.getGroup('sfp_tx', (
    ('txp', 'SFP_TX_P'),
    ('txn', 'SFP_TX_N'),
))
p.getGroup('sfp_rx', (
    ('rxp', 'SFP_RX_P'),
    ('rxn', 'SFP_RX_N')
))
p.getGpios('sfp_tx_disable_n', 'SFP_TX_DISABLE')
p.getGpios('sfp_rx_los', 'SFP_LOS_LS')

("sfp", 0,
    Subsignal("txp", Pins("AM4")),
    Subsignal("txn", Pins("AM3")),
    Subsignal("rxp", Pins("AL6")),
    Subsignal("rxn", Pins("AL5")),
),
("sfp_tx", 0,
    Subsignal("txp", Pins("AM4")),
    Subsignal("txn", Pins("AM3")),
),
("sfp_rx", 0,
    Subsignal("rxp", Pins("AL6")),
    Subsignal("rxn", Pins("AL5")),
),
("sfp_tx_disable_n", 0, Pins("AP33"), IOStandard("LVCMOS18")),
("sfp_rx_los", 0, Pins("BB38"), IOStandard("LVCMOS18")),


In [327]:
p.props["SDIO_DAT3_LS"] = p.props["SDIO_CD_DAT3_LS"]
p.keys = natsorted(p.props.keys())
p.getGroup('mmc', (
    ('clk', 'SDIO_CLK_LS'),
    ('cmd', 'SDIO_CMD_LS'),
    ('det', 'SDIO_SDDET'),
    ('wp', 'SDIO_SDWP'),
    ('dat', 'SDIO_DAT[0-3]_LS', 4),
))

("mmc", 0,
    Subsignal("clk", Pins("AN30"), IOStandard("LVCMOS18")),
    Subsignal("cmd", Pins("AP30"), IOStandard("LVCMOS18")),
    Subsignal("det", Pins("AP32"), IOStandard("LVCMOS18")),
    Subsignal("wp", Pins("AR32"), IOStandard("LVCMOS18")),
    Subsignal("dat", Pins("AR30 AU31 AV31 AT30"), IOStandard("LVCMOS18")),
),


In [330]:
p.getGpios('vadj_on_b', 'FMC_VADJ_ON_B_LS')
p.getConnector('FMC1_HPC')

("vadj_on_b", 0, Pins("AH35"), IOStandard("LVCMOS18")),
("FMC1_HPC", {
    "CLK0_M2C_N": "L40",
    "CLK0_M2C_P": "L39",
    "CLK1_M2C_N": "M31",
    "CLK1_M2C_P": "N30",
    "DP0_C2M_N": "E1",
    "DP0_C2M_P": "E2",
    "DP0_M2C_N": "D7",
    "DP0_M2C_P": "D8",
    "DP1_C2M_N": "D3",
    "DP1_C2M_P": "D4",
    "DP1_M2C_N": "C5",
    "DP1_M2C_P": "C6",
    "DP2_C2M_N": "C1",
    "DP2_C2M_P": "C2",
    "DP2_M2C_N": "B7",
    "DP2_M2C_P": "B8",
    "DP3_C2M_N": "B3",
    "DP3_C2M_P": "B4",
    "DP3_M2C_N": "A5",
    "DP3_M2C_P": "A6",
    "DP4_C2M_N": "J1",
    "DP4_C2M_P": "J2",
    "DP4_M2C_N": "H7",
    "DP4_M2C_P": "H8",
    "DP5_C2M_N": "H3",
    "DP5_C2M_P": "H4",
    "DP5_M2C_N": "G5",
    "DP5_M2C_P": "G6",
    "DP6_C2M_N": "G1",
    "DP6_C2M_P": "G2",
    "DP6_M2C_N": "F7",
    "DP6_M2C_P": "F8",
    "DP7_C2M_N": "F3",
    "DP7_C2M_P": "F4",
    "DP7_M2C_N": "E5",
    "DP7_M2C_P": "E6",
    "GBTCLK0_M2C_C_N": "A9",
    "GBTCLK0_M2C_C_P": "A10",
    "GBTCLK1_M2C_C_N": "E9",
    "

In [128]:
p.getConnector('FMC2_HPC')

("FMC2_HPC", {
    "CLK0_M2C_N": "AF40",
    "CLK0_M2C_P": "AF39",
    "CLK1_M2C_N": "T39",
    "CLK1_M2C_P": "U39",
    "DP0_C2M_N": "N1",
    "DP0_C2M_P": "N2",
    "DP0_M2C_N": "P7",
    "DP0_M2C_P": "P8",
    "DP1_C2M_N": "M3",
    "DP1_C2M_P": "M4",
    "DP1_M2C_N": "N5",
    "DP1_M2C_P": "N6",
    "DP2_C2M_N": "L1",
    "DP2_C2M_P": "L2",
    "DP2_M2C_N": "L5",
    "DP2_M2C_P": "L6",
    "DP3_C2M_N": "K3",
    "DP3_C2M_P": "K4",
    "DP3_M2C_N": "J5",
    "DP3_M2C_P": "J6",
    "DP4_C2M_N": "U1",
    "DP4_C2M_P": "U2",
    "DP4_M2C_N": "W5",
    "DP4_M2C_P": "W6",
    "DP5_C2M_N": "T3",
    "DP5_C2M_P": "T4",
    "DP5_M2C_N": "V3",
    "DP5_M2C_P": "V4",
    "DP6_C2M_N": "R1",
    "DP6_C2M_P": "R2",
    "DP6_M2C_N": "U5",
    "DP6_M2C_P": "U6",
    "DP7_C2M_N": "P3",
    "DP7_C2M_P": "P4",
    "DP7_M2C_N": "R5",
    "DP7_M2C_P": "R6",
    "GBTCLK0_M2C_C_N": "K7",
    "GBTCLK0_M2C_C_P": "K8",
    "GBTCLK1_M2C_C_N": "T7",
    "GBTCLK1_M2C_C_P": "T8",
    "HA00_CC_N": "AC33",
    "H